In [1]:
import random
from datetime import datetime, timedelta
import io
import numpy as np
import heapq
import json
import operator
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from stop_words import get_stop_words

import numpy.random as nprnd

stop_words = get_stop_words('english')


In [27]:
path_to_data = '../data/'

##########################
# load files #                           
##########################

training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0)
#training_info = pd.read_csv(path_to_data + 'training_info.csv', sep=',', header=0)
training_info = pd.read_csv(path_to_data+"training_info2.csv",sep=',', header=0, index_col=0)
test = pd.read_csv(path_to_data + 'test_set.csv', sep=',', header=0)
#test_info = pd.read_csv(path_to_data + 'test_info.csv', sep=',', header=0)
test_info = pd.read_csv(path_to_data+"test_info2.csv",sep=',', header=0, index_col=0)

In [28]:
# Correct dates and put datetime format
# We do that because we noticed test_set is only composed of email posterior to the ones of train_set. 
# Datetime format allows to simulate posteriority in our train/test split
from datetime import datetime

for row in training_info.sort(['date']).iterrows():
    date = row[1]['date']
    if date[:3] == '000':
        date = '2' + date[1:]
        
    training_info.loc[row[0], 'date'] = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

for row in test_info.sort(['date']).iterrows():
    date = row[1]['date']
        
    test_info.loc[row[0], 'date'] = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

/Users/christophelanternier/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/christophelanternier/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


### Functions

In [112]:
t1 = datetime.now()
dic_recency = {}
df_last_sent_emails = X_train_info_sender.sort_values(by = 'date', ascending = False)[:n_recency_features]
for row in df_last_sent_emails.iterrows():
    recipients = row[1]['recipients'].split()
    for recipient in recipients:
        if '@' in recipient:
            dic_recency[recipient] = dic_recency.get(recipient, 0) + 1
norm = sum(dic_recency.values())
for k,v in dic_recency.iteritems():
    dic_recency[k] = float(v)/norm
print datetime.now()-t1

0:00:00.007402


In [202]:
def most_similar_sklearn(array_embedding_sparse, mail_tfidf, n):
    
    similarities = cosine_similarity(array_embedding_sparse, mail_tfidf)
    if int(round(sorted(similarities[:,0], reverse=True)[0])) ==1:
        closest_ids = similarities[:,0].argsort()[::-1][1:]
    else:
        closest_ids = similarities[:,0].argsort()[::-1]
    
    return closest_ids[:n], similarities

def get_sender(query_mid, training):
    for row in training.iterrows():
        mids = row[1]['mids'].split()
        for mid in mids:
            if int(mid) == query_mid:
                sender = row[1]['sender']
                break
    return sender

def get_n_closest_emails(sender, n, closest_ids, training, training_info, mail_date, anterior = True):
    # The anterior option allows to consider only the closest emails that are anterior to the one considered
    
    # Get all emails' mids from query sender
    #all_emails_from_sender_mids = [int(k) for k in training[training['sender']==sender]['mids'].values[0].split()]

    # Get emails' index from query sender
    if anterior:
        training_info_anterior = training_info[training_info['date'] <= mail_date]
    else:
        training_info_anterior = training_info
        
    all_emails_from_sender_ids = training_info_anterior[training_info_anterior['mid'].isin(all_emails_from_sender_mids)].index.values

    # Get the closest emails WRITTEN BY THE SENDER
    closest_ids_per_sender = []
    for idx in closest_ids:
        if idx in all_emails_from_sender_ids:
            closest_ids_per_sender.append(idx)
        if len(closest_ids_per_sender) == n:
            break
            
    return closest_ids_per_sender, all_emails_from_sender_mids

def get_10_recipients(closest_ids_per_sender, training_info, similarities):
    dic_of_recipients = {}
    #weight = len(closest_ids_per_sender)+1
    for idx in closest_ids_per_sender:
        recipients = training_info.loc[idx,'recipients'].split()
        for recipient in recipients:
            if '@' in recipient:
                dic_of_recipients[recipient] = dic_of_recipients.get(recipient, 0) + similarities[idx][0]

    return dic_of_recipients

def get_recency_features(X_train_info_sender, mail_date, n_recency_features):    
    dic_recency = {}
    df_last_sent_emails = X_train_info_sender[X_train_info_sender.date<= mail_date].sort_values(by = 'date', ascending = False)[:n_recency_features]
    for row in df_last_sent_emails.iterrows():
        recipients = row[1]['recipients'].split()
        for recipient in recipients:
            if '@' in recipient:
                dic_recency[recipient] = dic_recency.get(recipient, 0) + 1
    norm = sum(dic_recency.values())
    for k,v in dic_recency.iteritems():
        dic_recency[k] = float(v)/norm
    
    return dic_recency

def mean_ap(suggested_10_recipients, ground_truth):
    MAP = 0
    correct_guess = 0
    for i, suggestion in enumerate(suggested_10_recipients):
        if suggestion in ground_truth:
            correct_guess +=1
            MAP += float(correct_guess)/(i+1)
    MAP = float(MAP)/min(10, len(ground_truth))
    return MAP

## Create Features

In [30]:
submission = False
training_info = training_info.sort_values(by='date')

if submission:
    # submission procedure
    X_train_info = training_info
    X_test_info = test_info
    
else:
    # test procedure
    split_date=datetime(2001, 8, 25)
    X_train_info = training_info[training_info.date <= split_date]
    #X_test_info = training_info[training_info.date > split_date]
    
    #Randomize selection of test set:
    X_test_info = training_info[training_info.date > split_date]
    mask = nprnd.choice(range(X_test_info.shape[0]), size=1000, replace=False)
    X_test_info.index = range(X_test_info.shape[0])
    X_test_info = X_test_info[X_test_info.index.isin(mask)]

    X_train_info = training_info

In [31]:
if submission:
    tfidf = TfidfVectorizer(stop_words = stop_words)
    array_embedding_sparse = tfidf.fit_transform(np.concatenate((X_train_info['body'].values,X_test_info['body'].values)))
    array_embedding_sparse = array_embedding_sparse[:X_train_info.shape[0]]
else:
    #With porter stemming:
    #tfidf = TfidfVectorizer(tokenizer= tokenize, stop_words = stop_words)
    #Without stemming:
    tfidf = TfidfVectorizer(stop_words = stop_words)
    array_embedding_sparse = tfidf.fit_transform(X_train_info['body'].values)

## Training

In [48]:
df = pd.read_csv('new_features_partial.csv')

In [204]:
all_mean_ap = []
all_ground_truth = []
all_suggestions = []
results = pd.DataFrame(columns=['recipients'])
results.index.name = 'mid'

index = 0
new_features_df = pd.DataFrame(columns=['recipients', 'KNNScore','NSF', 'NRF','recency', 'label'])
# number of closest neighbors to collect recipients from:
n = 30
n_recency_features = 50

#re-arrange train index
X_train_info.index = range(X_train_info.shape[0])

t_all = datetime.now()

count = 1
query_id = 19516
for query_id in [X_train_info.index.values[10000]]:

    count+=1
    if count%100==0:
        print count

    # STEP 1: preliminary assignment
    t1 = datetime.now()
    mail = X_train_info['body'][query_id]
    mail_date = X_train_info['date'][query_id]
    query_mid = X_train_info['mid'][query_id]
    ground_truth = X_train_info['recipients'][query_id].split()

    mail_tfidf = tfidf.transform([mail])
    t_step1 = datetime.now()-t1
    print "STEP 1 took ", t_step1
    # END OF STEP 1

    # STEP 2: get sender
    t2 = datetime.now()
    sender = X_train_info[X_train_info.mid == query_mid]['sender'].values[0]
    X_train_info_sender = X_train_info[X_train_info.sender == sender]
    index_sender = X_train_info_sender.index.values
    X_train_info_sender.index = range(X_train_info_sender.shape[0])
    array_embedding_sparse_sender = array_embedding_sparse[index_sender]
    t_step2 = datetime.now() - t2
    print "STEP 2 took ", t_step2
    # END OF STEP 2 --- > divided by 100

    # STEP 3: Rank mails by cosine similarity with searched mail
    t3 = datetime.now()
    closest_ids_per_sender, similarities = most_similar_sklearn(array_embedding_sparse_sender, mail_tfidf, n)
    t_step3 = datetime.now() - t3
    print "STEP 3 took ", t_step3
    # END OF STEP 3

    # STEP 5: compute recency features
    t5 = datetime.now()
    dic_recency = get_recency_features(X_train_info_sender, mail_date, n_recency_features)
    t_step5 = datetime.now() - t5
    print "STEP 5 took ", t_step5
    # END OF STEP 5

    # STEP 7: Create dictionnary of all recipient for the 30 most similar emails, and get frequency
    t7 = datetime.now()
    dic_of_recipients = get_10_recipients(closest_ids_per_sender, X_train_info_sender, similarities)
    t_step7 = datetime.now() - t7
    print "STEP 7 took ", t_step7
    # END OF STEP 7

    # STEP 8: Create new features
    t8 = datetime.now()
    for k,v in dic_of_recipients.iteritems():
        KNNScore = v
        NSF = sent_to[sender][k]
        if sender in received_from.keys():
            if k in received_from[sender].keys():
                NRF = received_from[sender][k]
            else:
                NRF = 0
        else:
            NRF = 0
        if k in ground_truth:
            label = 1
        else:
            label = 0
        if k in dic_recency.keys():
            recency = dic_recency[k]
        else:
            recency = 0
        new_features_df.loc[index] = [k, KNNScore, NSF, NRF, recency, label]
        index +=1

    t_step8 = datetime.now() - t8
    print "STEP 8 took ", t_step8
    # END OF STEP 8

print "total took:", datetime.now()-t_all

STEP 1 took  0:00:00.002935
STEP 2 took  0:00:00.006111
STEP 3 took  0:00:00.001176
STEP 5 took  0:00:00.008084
STEP 7 took  0:00:00.016337
STEP 8 took  0:00:01.136626
total took: 0:00:01.172883


In [196]:
len(dic_of_recipients)

317

In [130]:
len(closest_ids_per_sender)

168

## Get NSF and NRF
- NSF: number of messages sent by sender to this recipient, divided by all messages sent by sender 
- NRF: number of messages received from the recipient to the sender, divided by all messages received by sender

In [26]:
X_train_info.shape

(43613, 4)

In [15]:
for row in training_info.iterrows():
    recipients = row[1]['recipients'].split()
    for recipient in recipients:
        if recipient == 'wsmith@wordsmith.org':
            print recipient

In [12]:
received_from['wsmith@wordsmith.org']

KeyError: 'wsmith@wordsmith.org'

In [7]:
sent_to = {}
for row in training.iterrows():
    sender = row[1]['sender']
    sent_to[sender] = {}
    for mid in row[1]['mids'].split():
        recipients = training_info[training_info.mid == int(mid)]['recipients'].values[0].split()
        for recipient in recipients:
            if '@' in recipient:
                if recipient in sent_to[sender].keys():
                    sent_to[sender][recipient] += 1
                else:
                    sent_to[sender][recipient] = 1

In [8]:
received_from = {}
for sender in sent_to.keys():
    for recipient in sent_to[sender].keys():
        if recipient not in received_from.keys():
            received_from[recipient] = {}
        if sender not in received_from[recipient].keys():
            received_from[recipient][sender] = sent_to[sender][recipient]
        else:
            received_from[recipient][sender] += sent_to[sender][recipient]

In [9]:
# normalization:
for sender in sent_to.keys():
    norm = sum(sent_to[sender].values())
    for recipient in sent_to[sender].keys():
        sent_to[sender][recipient] = float(sent_to[sender][recipient])/norm

In [10]:
# normalization:
for recipient in received_from.keys():
    norm = sum(received_from[recipient].values())
    for sender in received_from[recipient].keys():
        received_from[recipient][sender] = float(received_from[recipient][sender])/norm